## Инструменты для решения задач на больших графах

In [1]:
# несколько удобных функций для описания датасетов
def describe_dataset(dataset):
    print(f'Dataset: {dataset}:')
    print('======================')
    print(f'Number of graphs: {len(dataset)}')
    print(f'Number of features: {dataset.num_features}')
    print(f'Number of classes: {dataset.num_classes}')

def describe_graph(g):
    print(g)
    print('==============================================================')

    # Gather some statistics about the graph.
    print(f'Number of nodes: {g.num_nodes}')
    print(f'Number of edges: {g.num_edges}')
    print(f'Average node degree: {g.num_edges / g.num_nodes:.2f}')
    if hasattr(g, 'train_mask'):
        print(f'Number of training nodes: {g.train_mask.sum()}')
        print(f'Training node label rate: {int(g.train_mask.sum()) / g.num_nodes:.2f}')
    print(f'Has isolated nodes: {g.has_isolated_nodes()}')
    print(f'Has self-loops: {g.has_self_loops()}')
    print(f'Is undirected: {g.is_undirected()}')

In [3]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='./tmp/Planetoid', name='PubMed', transform=NormalizeFeatures())
g = dataset[0]
describe_dataset(dataset)
describe_graph(g)

Dataset: PubMed():
Number of graphs: 1
Number of features: 500
Number of classes: 3
Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Number of nodes: 19717
Number of edges: 88648
Average node degree: 4.50
Number of training nodes: 60
Training node label rate: 0.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Cluster-GCN работает следующим образом. Сначала граф делится на подграфы при помощи алгоритмов выделения сообществ. После этого, связи между подграфами удаляются, и операцию свертки становится возможным произвести без опасений столкнуться с нехваткой памяти из-за большого кол-ва соседей, которые требуется обработать (neighborhood explosion).

Из-за того, что такое удаление связей может негативно сказаться на модели, Cluster-GCN на этапах создания пакетов данных добавляет связи между кластерами узлов (stochastic partitioning):

![Cluster-GCN](assets/cluster-gcn-partitioning.png)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric.nn as gnn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(
        self, 
        n_input: int, 
        n_hidden_layers: int, 
        n_hidden: int, 
        n_out: int,
        dropout_p: float = 0.2,
        activation: callable = F.relu
    ) -> None:
        super().__init__()
        self.dropout_p = dropout_p
        self.layers = nn.ModuleList()
        if n_hidden_layers > 1:
            self.layers.append(gnn.GCNConv(n_input, n_hidden))
            for _ in range(1, n_hidden_layers-1):
                self.layers.append(gnn.GCNConv(n_hidden, n_hidden))
            self.layers.append(gnn.GCNConv(n_hidden, n_out))
        else:
            self.layers.append(gnn.GCNConv(n_input, n_out))

        self.activation = activation

    def forward(self, x, edge_index):
        h = x
        for idx, layer in enumerate(self.layers):
            h = layer(h, edge_index)
            if idx != len(self.layers) - 1:
                h = self.activation(h)
                h = F.dropout(h, p=self.dropout_p, training=self.training)
        return h

In [12]:
from torch_geometric.loader import ClusterData, ClusterLoader
# 1. При помощи ClusterData конвертируем граф в датасет подграфов
cluster_g = ClusterData(g, num_parts=128)
# 2. Создаем лоадер для стохастического разбиения:
train_loader = ClusterLoader(cluster_g, batch_size=32, shuffle=True)
# 3. Теперь берем любую модель для классификации узлов и применяем ее, но
# вместо обработки всего графа сразу обрабатываем узлы в мини-пакетном режиме
model = GCN(
    n_input=dataset.num_features, 
    n_hidden_layers=2, 
    n_hidden=16,
    n_out=dataset.num_classes, 
    activation=torch.relu,
    dropout_p=0.5,
)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01, weight_decay=5e-4)

for epoch in range(51):
    for clust_batch in train_loader:
        logits = model(clust_batch.x, clust_batch.edge_index)
        loss = criterion(logits[clust_batch.train_mask], clust_batch.y[clust_batch.train_mask])
        loss.backward()  
        optimizer.step()
        optimizer.zero_grad()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss.item():.4f}')

preds = model(g.x, g.edge_index).argmax(dim=1)
accs = []
for mask in [g.train_mask, g.val_mask, g.test_mask]:
    correct = preds[mask] == g.y[mask]  
    accs.append(int(correct.sum()) / int(mask.sum()))  
print(f'Train Acc: {accs[0]:.4f}, Val Acc: {accs[1]:.4f}, Test Acc: {accs[2]:.4f}')

Computing METIS partitioning...
Done!


GCN(
  (layers): ModuleList(
    (0): GCNConv(500, 16)
    (1): GCNConv(16, 3)
  )
)
Epoch: 000, Loss: 1.0853
Epoch: 010, Loss: 0.9542
Epoch: 020, Loss: 0.5547
Epoch: 030, Loss: 0.3817
Epoch: 040, Loss: 0.2095
Epoch: 050, Loss: 0.1536
Train Acc: 0.9667, Val Acc: 0.7540, Test Acc: 0.7350
